# 자치구 별 쓰레기통 커버리지 면적 구하기

In [ ]:
import math
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon
from shapely.ops import cascaded_union

# 위도, 경도를 이용해 유사 원(정 360각형)을 Polygon 데이터로 만듦

# pi는 math.acos(-1)를 사용해도 무방
def calc_offsets(d, lat):
    return [
        abs(360*math.asin(math.sin(d/6371/2/1000)/math.cos(lat*math.pi/180))/math.pi),
        180*d/6371/1000/math.pi
    ]

def radian(degree):
    return math.acos(-1)/180*degree

def coordinate_after_rotation(c, degree, offsets):
    return [
        c[0]+math.cos(radian(degree))*offsets[0],
        c[1]+math.sin(radian(degree))*offsets[1]
    ]

def make_circle(lng, lat):
    c = (lng, lat)
    d = 100

    rotating_degree = 1

    offsets = calc_offsets(d, c[1])
    coordinates = [coordinate_after_rotation(c, d, offsets) for d in range(0, 360, rotating_degree)]
    
    return Polygon(coordinates)

temp_circle = make_circle(126.973210, 37.576)
temp_circle

In [ ]:
# 쓰레기통 위치 정보를 가져 옴
import pandas as pd

addr = pd.read_excel('서울시 가로 쓰레기통 위치 위도, 경도.xlsx')
addr = addr.drop(['연번'], axis =1)
addr

In [ ]:
# 우리나라 전체 읍면구 데이터를 shape 파일로 가져와서 서울시 데이터만 이용(140~164)
temp = gpd.GeoDataFrame.from_file('읍면구\TL_SCCO_SIG.shp', encoding='cp949')

temp

In [5]:
# 서울시 데이터만 뽑아 냄
seoul_df = temp
land_area = dict()

for i in range(140,165):
    name = temp['SIG_KOR_NM'][i]
    t_t = temp['geometry'][i]
    t = temp['geometry'][i].area
    land_area[name] = t

In [ ]:
# 쓰레기통 위치가 어느 자치구에 있는지 구분해 줌

idx_dict = dict()
circles = []

for i in range(len(addr)):
    loc = addr['설치위치'][i].split()[0]
    
    if(not loc in idx_dict):
        idx_dict[loc] = []
    
    idx_dict[loc].append(i)
    
for i in range(len(addr)):
    circles.append(make_circle(addr['경도'][i], addr['위도'][i]))

idx_dict

In [7]:
# QGIS를 이용해 Geometry데이터를 위도 경도로 변환해 불러온다.
import json

json_temp = []

for line in open('./data.geojsonl.json', 'r', encoding='UTF-8'):
    json_temp.append(json.loads(line))

In [31]:
# 전체 면적 - 쓰레기통 커버 면적
result_area = []
# (전체 면적 - 쓰레기통 커버 면적) * 100 / 전체 면적
result_percent = []

# 140 ~ 165까지가 서울시 자치구
for i in range(140, 165):
    # 자치구 Polygon data
    poly_temp = Polygon([[p[0], p[1]] for p in json_temp[i]['geometry']['coordinates'][0]])
    # 자치구 이름
    name = json_temp[i]['properties']['SIG_KOR_NM']
    # 해당 자치구에 있는 쓰레기통 커버(100m 반지름의 원)들을 모아 놓음
    polygons = [circles[i] for i in idx_dict[name]]
    # 모은 polygon들을 하나의 Multi Polygon 데이터로 만들어 준다.
    temp_trash = cascaded_union(polygons)
    # 전체 면적에서 쓰레기통 커버 면적을 빼준다.
    trash = gpd.GeoSeries(temp_trash)
    df1 = gpd.GeoDataFrame({'geometry': poly_temp, 'df1':[1.2]})
    df2 = gpd.GeoDataFrame({'geometry': trash, 'df2':[1]})
    result = gpd.overlay(df1, df2, how='difference')
    result_area.append(result['geometry'][0].area*(10**10))
    result_percent.append((temp_trash.area / poly_temp.area) * 100)

In [36]:
# 결과값을 dataframe에 저장
result_df = pd.DataFrame({
    '자치구': list(land_area.keys()),
    '결과 면적': result_area,
    '수요충족비율 -> (쓰레기통 커버리지 / 자치구 면적 (%))': result_percent
})

In [38]:
# 결과 dataframe을 엑셀 파일로 저장
result_df.to_excel('수요충족비율_최종.xlsx')